<a href="https://colab.research.google.com/github/jalew188/PeptDeep-HLA/blob/master/nbs/HLA1_transfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transfer learning of sample-specific HLA-I models

> To enable GPU in colab, click `Runtime -> Change runtime type`.

In [1]:
%pip install git+https://github.com/MannLabs/PeptDeep-HLA.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/MannLabs/PeptDeep-HLA.git to /tmp/pip-req-build-wrej0ou6
  Running command git clone -q https://github.com/MannLabs/PeptDeep-HLA.git /tmp/pip-req-build-wrej0ou6


In [2]:
%pip install gdown

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import gdown

fasta_url = "https://drive.google.com/file/d/1V9KxDniKwZFZnHlP58EbjkuelNJnp1Kq/view?usp=share_link"
fasta = 'UP000005640_human_reviewed.fasta'

gdown.download(fasta_url, fasta, fuzzy=True)

Downloading...
From: https://drive.google.com/uc?id=1V9KxDniKwZFZnHlP58EbjkuelNJnp1Kq
To: /content/UP000005640_human_reviewed.fasta
100%|██████████| 13.6M/13.6M [00:00<00:00, 160MB/s]


'UP000005640_human_reviewed.fasta'

#### Load training HLA peptides

It could be a tsv/csv/txt file containing sample-specific HLA-I peptides. It must contain 'sequence' column. 

In [4]:
import pandas as pd
train_seq_df = pd.DataFrame({
    'sequence': [
        'ACDEFGHIKLMNPQ',
        'ACDEFGHI',
        'ACDEFGHIK',
        'EFGHIKLMNPQ',
        'AHIKLMNPQ',
    ]
})
train_seq_df['nAA'] = train_seq_df.sequence.str.len()
train_seq_df

,sequence,nAA
0,ACDEFGHIKLMNPQ,14
1,ACDEFGHI,8
2,ACDEFGHIK,9
3,EFGHIKLMNPQ,11
4,AHIKLMNPQ,9


#### Initialize the model and load the pretrained model

In [5]:
from peptdeep_hla.HLA_class_I import HLA_Class_I_Classifier
model = HLA_Class_I_Classifier(
    fasta_files=[fasta]
)
model.get_parameter_num()

1669697

In [6]:
from peptdeep_hla.HLA_class_I import pretrained_HLA1
model.load(pretrained_HLA1)
pretrained_HLA1

'/usr/local/lib/python3.8/dist-packages/peptdeep_hla/pretrained_models/HLA1_IEDB.pt'

#### Train by the training peptides

The non-HLA peptides are automatically sampled from the fasta file as the negative training data.

In [7]:
model.train(
    train_seq_df, 
    epoch=40, warmup_epoch=10, 
    verbose=True
)

[Training] Epoch=1, lr=5e-05, loss=1.484032779932022
[Training] Epoch=2, lr=0.0001, loss=1.4480046033859253
[Training] Epoch=3, lr=5e-05, loss=0.8671623691916466
[Training] Epoch=4, lr=0.0, loss=0.3142213933169842


#### Predict HLA-I peptides from fasta

In [ ]:
hla_df = model.predict_from_proteins(prob_threshold=0.7)
hla_df

/usr/local/lib/python3.8/dist-packages/pydivsufsort/divsufsort.py:76: UserWarning: converting str argument uses more memory
  warnings.warn("converting str argument uses more memory")
/usr/local/lib/python3.8/dist-packages/alphabase/protein/lcp_digest.py:7: UserWarning: converting str argument uses more memory
  lcp_array = kasai(cat_prot, suffix_array)
 12%|█▎        | 9/72 [2:13:16<15:29:20, 885.09s/it]

In [ ]:
hla_df.to_csv('Predicted_HLA.tsv',index=False, sep="\t")

To download `Predicted_HLA.tsv`, click the `Files` (folder logo) in the left panel and choose the file.